# ADM-HW3
---

## 1. Data Collection

Here are the imports needed for the following sections of code.

In [2]:
import random
import re
import time
import json

import requests
from bs4 import BeautifulSoup

In [7]:
data_path = r"C:\Users\paolo\Documents\Data Science\ADM\data"

### 1.1. Get the list of movies

This first subtask consists in parsing the html page and getting the complete list of urls.
For this task we created the following function:

In [3]:
def get_movies_links(filepath):
    """
    Takes in input the filepath of the html file containing the links of the pages to download,
    parses it and returns a list containing the links.
    :param filepath: path of the html file containing all the links of the wikipedia pages
    :return: list of links
    """
    with open(filepath, "r", encoding='utf-8') as f:
        html = f.read()
    soup = BeautifulSoup(html, 'html.parser')

    links = []
    for a in soup.select('a'):
        links.append(a.get('href'))

    return links

### 1.2. Crawl Wikipedia

In this section we are going to crawl specific informations from each wikipedia page.

First of all we need a function to download the pages.
During the download process, this function also creates a dict to associate to each movie its url, and then stores it in a json file.

In [4]:
def download_pages(links, index_to_link_path):
    """
    Iterate over the list of links, downloads each html page and stores it physically.
    :param links: list of links of wikipedia pages of movies
    :return: None
    """

    # opening/creating a json file that associates to each file the respective link
    try:
        with open(index_to_link_path, 'r') as f:
            index_to_link = json.load(f)
    except FileNotFoundError:
        index_to_link = {}

    i = 0
    while i < len(links):
        try:
            response = requests.get(links[i])
            filename = "article_" + str(i) + ".html"
            filepath = "data/movies/" + filename
            if response.status_code == 200:
                with open(filepath, "w", encoding='utf-8') as f:
                    f.write(response.text)
                    # create a file which associates to each movie the url. We will need it for the final print of the results table
                    index_to_link[i] = links[i]
            i += 1
            # wait a random interval of seconds before performing the next request
            t = random.randint(1, 5)
            time.sleep(t)
        except Exception as e:
            time.sleep(60 * 20)
            print(e)

    with open(index_to_link_path, 'w') as out:
        json.dump(index_to_link, out)

The collection functions for getting the links and downloading the pages have been grouped in the **collector_utils.py** file. In the **collector.py** has been created the function *collect()* that makes the collection process start.

In [6]:
from utils import collector_utils as cu


def collect(html_filepath, index_to_link_path):
    """
    Uses the functions in collector_utils to retrieve and download the wikipedia pages
    :param html_filepath: path of the html file containing all the links to the wikipedia pages
    :return: None
    """
    links = cu.get_movies_links(html_filepath)
    cu.download_pages(links, index_to_link_path)

Before starting the collection process we need to define some paths.

In [8]:
movies_html_path = data_path + r'\movies1.html'
index_to_link_path = data_path + r'\index_to_link.json'

In [ ]:
collector.collect(movies_html_path, index_to_link_path)

In case the movie to url association hasn't been made during the downloading process, here is a function to create the json file afterwards.

In [10]:
def associate_urls(movies_path, index_to_link_path):
    """
    Creates a json file that associates to the index of each movie the relative url.
    :param movies_path: path of the folder containing the movies html files.
    :param index_to_link_path: path to the json file where to store the urls.
    :return:
    """
    import os

    movies = os.listdir(movies_path)
    try:
        with open(index_to_link_path, 'r') as f:
            index_to_link = json.load(f)
    except FileNotFoundError:
        index_to_link = {}

    for movie in movies:
        i = re.findall(r'\d+', movie)[0]
        if i in index_to_link:
            continue
        print(i)
        movie_path = movies_path + '/' + movie
        with open(movie_path, "r", encoding='utf-8') as f:
            html = f.read()
        soup = BeautifulSoup(html, 'html.parser')
        canonical_links = soup.find_all("link", {"rel": "canonical"})
        url = canonical_links[0].get('href')
        index_to_link[i] = url

    with open(index_to_link_path, 'w') as out:
        json.dump(index_to_link, out)

### 1.3 Parse downloaded pages

At this point we have downloaded and stored all the wikipedia pages.
Now it's time to parse each of them and store the required informations into tsv files.

For each page we need to parse **Title**, **Intro**, **Plot**, **Infobox**. Here are the functions to do it. All of them just take in input the *soup* for the html page we need to parse.

In [11]:
def get_title(soup):
    """
    :param soup:
    :return: cleaned up title of the movie page
    """
    import re

    title = soup.title.text
    # most page titles contain the '- Wikipedia' substring, so we are going to remove it.
    title = title.replace('- Wikipedia', '')
    # Another substring pattern repeated a lot of times is the presence of parentheses generally
    # containing the substring 'film', or the year of creation (which we are going to extrapolate
    # later from the infobox. So we are going to remove them too.
    title = re.sub('\(.*?\)', '', title)
    # As finishing touch, we are going to remove any possible extra spaces at the beginning and end.
    title = title.strip()
    return title

In [12]:
def get_intro(soup):
    """
    Retrieves and returns the intro, from the html page, which is generally
    composed by all the html paragraphs before the first not <p> tag.
    :param soup
    :return: intro of the movie page
    """
    infobox = soup.find('table', class_='infobox')
    if infobox is not None:
        p = infobox.find_next_sibling('p')
    else:
        p = soup.p
    intro = p.text
    next_sib = p.find_next_sibling()
    while next_sib and next_sib.name == 'p':
        intro += next_sib.text
        next_sib = next_sib.find_next_sibling()
    return intro

In [13]:
def get_plot(soup):
    """
    Retrieves and returns the plot, from the html page, which is generally
    composed by all the html paragraphs after the first h tag containing the "Plot" string in it
    and before the next tag different from <p>.
    :param soup
    :return: plot of the movie page
    """
    plot = ""
    h_list = soup.find_all(['h1', 'h2', 'h3', 'h4'])
    for h in h_list:
        if 'Plot' in h.text or 'plot' in h.text:
            plot_h = h
            siblings = plot_h.find_next_siblings()
            for sib in siblings:
                if sib.name != 'p':
                    break
                plot += sib.text
            break
    if plot == "":
        return "NA"
    return plot

For the infobox we also need to create a *dict* which associates to each infobox field the name of the field which will be used in the tsv file.

In [14]:
infobox_keys = {
    "Directed by": "director",
    "Produced by": "producer",
    "Written by": "writer",
    "Starring": "starring",
    "Music by": "music",
    "Release date": "release date",
    "Running time": "runtime",
    "Country": "country",
    "Language": "language",
    "Budget": "budget"
}

We also need a function to split the names which result unified after parsing them (e.g. GarwoodMarguerite -> Garwood Marguerite).

In [16]:
def split_names(s):
    """
    Takes a string generally obtained from the infobox and splits it if the are
    names unified (e.g. GarwoodMarguerite -> Garwood Marguerite)
    :param s: string
    :return: new string
    """
    for i in range(1, len(s)):
        if s[i].isupper() and s[i - 1] != ' ':
            s = s[:i] + ' ' + s[i:]
    return s

In [18]:
def get_infobox(soup):
    """
    :param soup
    :return: dictionary containing the title of the movie, a list of fields and a list of values
    """
    infobox_dict = dict()
    infobox = soup.find('table', class_='infobox')
    infobox_dict['fields'] = []
    infobox_dict['values'] = []

    if infobox is not None:
        # find and save the film_name
        infobox_dict['fields'].append('film_name')
        if infobox.th is not None:
            infobox_dict['values'].append(infobox.th.text)
        else:
            # if not find, save NA
            infobox_dict['values'].append('NA')

        # find all the rows of the infobox
        info = infobox.find_all('tr')[2:]

        for tr in info:
            # for each row, check if its th is note None and if the field is needed
            # then eventually store it
            if (tr.th is not None) and (tr.th.text in infobox_keys.keys()):
                field = infobox_keys[tr.th.text]
                infobox_dict['fields'].append(field)
                infobox_dict['values'].append(split_names(tr.td.text))

        # for all the field needed but not found, set their value as NA
        for f in infobox_keys.values():
            if f not in infobox_dict['fields']:
                infobox_dict['fields'].append(f)
                infobox_dict['values'].append('NA')

    return infobox_dict

We also need a function that takes in input all the info for a movie and stores them into a tsv file.

In [19]:
def save_tsv(title, intro, plot, infobox, filepath):
    import csv
    """
    Saves a .tsv file containing title, intro, plot and infobox.
    :param intro:
    :param plot:
    :param infobox:
    :param filepath:
    :return: None
    """
    fields = ['title', 'intro', 'plot'] + infobox['fields']
    values = [title, intro, plot] + infobox['values']
    with open(filepath, 'w', encoding='utf-8') as out:
        tsv_writer = csv.writer(out, delimiter='\t')
        tsv_writer.writerow(fields)
        tsv_writer.writerow(values)

All of the previous functions from 1.3 have been grouped into **parser_utils.py**.
They are called from the next function, *parse_file*, saved into **parser.py**.

In [20]:
from utils.parser_utils import *
from bs4 import BeautifulSoup


def parse_file(filepath, i, tsv_path):
    """
    Takes in input the filepath and the index of the file to parse and creates a tsv file
    containing title, intro, plot and infobox information taken from the html.
    :param filepath: path of the html file to parse
    :param i: index which identifies the file
    :return: None
    """

    with open(filepath, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f.read(), 'html.parser')
    title = get_title(soup)
    intro = get_intro(soup)
    plot = get_plot(soup)
    infobox = get_infobox(soup)

    out_path = tsv_path + '/info_' + i + '.tsv'
    save_tsv(title, intro, plot, infobox, out_path)

#### Multithread process

To make the html parsing and tsv creation process faster, we decided to use multithread programming by creating the following function.

In [21]:
def multithread_create_tsv_files(movies_path, tsv_path, n_threads):
    """
    This functions operates in multithread. It subdivides the list of files to parse into chucks and
    let each thread call the function create_tsv_files over each chunk.
    :param movies_path:
    :return:
    """
    # list all the files to parse and divide the list into chunks
    # depending on the number of threads to use.
    files = os.listdir(movies_path)
    l_chunks = N // n_threads
    chunks = [files[i:i + l_chunks] for i in range(0, N, l_chunks)]

    if len(chunks) > n_threads:
        chunks[-2] += chunks[-1]
        del chunks[-1]

    threads = []

    start_time = time.time()
    for i in range(n_threads):
        # create n_threads threads, each one of each calls the create_tsv_files function
        # on a different chunk of files.
        threads.append(
            Thread(name='Thread-' + str(i), target=create_tsv_files, args=[movies_path, tsv_path, chunks[i]]))

    # start each thread
    for t in threads:
        t.start()

    # wait for each thread to finish its execution
    for t in threads:
        t.join()

    minutes = (time.time() - start_time) / 60
    print("---------------- TSV CREATED in %s minutes ------------------" % minutes)

By running the next line of code, the parsing process starts.

In [ ]:
movies_path = data_path + r'\movies'
tsv_filepath = data_path + r'\tsv_files'
multithread_create_tsv_files(movies_path, tsv_filepath, n_threads=6)

## Search Engine

Now, we want to create two different Search Engines that, given as input a query, return the movies that match the query.

### 2.1. Conjunctive query

At this moment, we narrow our interest on the *intro* and *plot* of each document. It means that the first Search Engine will evaluate queries with respect to the aforementioned information.

For the homework it has been requestd to create a **vocabulary** file that associates to each word a term_id. We created the functions to that, but at the end we decided to not use them and stick with the actual word to creat the index, since the difference in time was minimal.

Here are the functions to create the *vocabulary.json* file.

In [22]:
def map_into_vocabulary(text, vocab):
    """
    Given a text and a vocab (dict), assigns to each word of the text and id.
    :param text:
    :param vocab:
    :return:
    """
    term_id = 0
    if len(vocab) > 0:
        keys = sorted(list(map(int, list(vocab.values()))), reverse=True)
        term_id = keys[0] + 1

    for word in text.split():
        if word not in vocab:
            vocab[word] = term_id
            term_id += 1


def map_vocabulary(vocab_filepath, tsv_filepath):
    """
    Performs the function map_into_vocabulary for each tsv file in tsv_filepath and then stores the vocab
    into a json file.
    :param vocab_filepath:
    :param tsv_filepath:
    :return:
    """
    files = os.listdir(tsv_filepath)

    try:
        with open(vocab_filepath, 'r') as f:
            vocab = json.load(f)
    except FileNotFoundError:
        vocab = {}

    for filename in files:
        print(filename)
        text = get_text(tsv_filepath, filename)
        map_into_vocabulary(text, vocab)

    with open(vocab_filepath, 'w') as out:
        json.dump(vocab, out)

The functions for the vocabulary creation have been grouped into the **index_utils.py** file.  
By running the next lines of code you create the vocabulary.

In [ ]:
index_utils.map_vocabulary(vocab_filepath, tsv_filepath)
print("---------------- Vocabulary Mapped --------------------")

#### 2.1.1) Create your index!

Here we are going to define the function used to create the first *inverted_index*, which will associate to each word the list of tsv documents which contain it. Differently from the suggestion in the README we will not use the term_id.

In [23]:
import json
import math
import os
import os.path

import pandas as pd

from utils import utils

First of all we need to define a function to clean the text after getting it from the tsv file.  
To the text will be performed the following cleaning operations:  
1) Removing stopwords  
2) Removing punctuation  
3) Stemming  

In [24]:
def clean_up(s):
    """
    Takes in input a string to be cleaned up.
    Returns a new cleaned up string.
    :param s
    :return: cleaned up string
    """
    import string

    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    from nltk.stem import PorterStemmer

    # Removing stopwords, punctuation and stemming
    stop_words = set(stopwords.words('english'))
    punctuation = set(string.punctuation)
    punctuation.add('``')
    ps = PorterStemmer()

    s = s.replace('\n', ' ')  # replacing new lines with a space
    word_tokens = word_tokenize(s)
    # Checking the each word is both not a stopword or punctuation and, after that,
    # stemming and lowering it. It is important to lower each word because otherwise,
    # in future steps, the search engine would consider for example "Rome" and "rome" two different words.
    filtered_string = [ps.stem(w.lower()) for w in word_tokens if w not in stop_words and w not in punctuation]

    return ' '.join(filtered_string)

The *clean_up* function has been stored in the **utils.py** file since it will be called from external funcions too.

To be able to create the index, we need the text of each tsv file, defined in this case as intro + plot. Here is the function to get the text, stored into **index_utils.py**.

In [26]:
def get_text(tsv_filepath, filename):
    """
    Given in input the path to the folder containing the tsv files and the name of the file to parse.
    Returns a string composed by intro + plot.
    :param tsv_filepath:
    :param filename:
    :return:
    """
    filepath = tsv_filepath + '/' + filename
    tsv = pd.read_csv(filepath, delimiter='\t')
    intro = tsv.loc[0, 'intro']
    plot = tsv.loc[0, 'plot']
    text = utils.clean_up(str(intro) + " " + str(plot))
    return text

Now we are able to create the first inverted index using the following function, stored into **index.py**. The index will be stored as *.json* file, so that we need to create it just one time.

In [27]:
def create_inverted_index(index_filepath, tsv_filepath):
    """
    Takes in input the filepath of the inverted inverted index and the filepath of the folder containing
    the tsv files.
    Creates a file at the index_filepath which associates to each word found in any document, the list of
    documents (tsv files) which contain it. Then stores the file on the drive.
    :param index_filepath:
    :param tsv_filepath:
    :return: None
    """
    # open the index file if already exists or create and index dict otherwise
    try:
        with open(index_filepath, 'r') as f:
            index = json.load(f)
    except FileNotFoundError:
        index = {}

    # list all the tsv files
    files = os.listdir(tsv_filepath)

    for filename in files:
        print(filename)
        text = index_utils.get_text(tsv_filepath, filename)

        for word in text.split():
            index[word] = index.get(word, []) + [filename]

    with open(index_filepath, 'w') as out:
        json.dump(index, out)

By running the next lines of code you create the inverted index.

In [ ]:
index_filepath = data_path + r'\index.json'
index = index.create_inverted_index(index_filepath, tsv_filepath)
print("---------------- Index Created --------------------")

#### 2.1.2) Execute the query

In [29]:
import json
import os
import os.path
import re
import time
from threading import Thread

import pandas as pd

Given a query, that you let the user enter, the Search Engine is supposed to return a list of documents. Since we are dealing with conjunctive queries (AND), each of the returned documents should contain all the words in the query.

In [28]:
def execute_query_base(index, query, index_to_link_path, tsv_path):
    # Creating a set of movies which contains all the words form the query
    movies = set()
    words_dict = dict()  # associates to each word the list of movies
    words = query.split()
    first_iter = True
    for word in words:
        if first_iter:
            movies.update(index.get(word, []))
            first_iter = False
            continue
        # with intersection_update we are able to maintain only the movies which contain all the word from the query
        movies.intersection_update(index.get(word, []))

    with open(index_to_link_path, 'r') as f:
        index_to_link = json.load(f)

    # create the final dict containing title, intro, url and similarity for each movie
    results = dict()
    for movie in movies:
        filepath = tsv_path + '/' + str(movie)
        tsv = pd.read_csv(filepath, delimiter='\t')
        # with open(filepath, 'r') as f:
        #     tsv = csv.DictReader(f, dialect='excel-tab')
        title = tsv.loc[0, 'title']
        intro = tsv.loc[0, 'intro']
        index = re.findall(r'\d+', movie)[0]
        url = index_to_link[str(index)]
        results[index] = [title, intro, url]

    # convert the dict to a pandas dataframe
    table = pd.DataFrame(results, columns=['Title', 'Intro', 'Wikipedia Url'])
    return table

By running the next lines of code you obtain and execute the query.

In [ ]:
query = utils.clean_up(input("Please, write a query: "))
table = execute_query_base(index, query, index_to_link_path, tsv_filepath)
print(table.to_string(index=False))

### 2.2) Conjunctive query & Ranking score

In this section is going to be defined a new search engine that, given a query, returns the *top-k* documents related to the query.

#### 2.2.1) Inverted index

First of all we need to create and *inverted_score_index*.  
This index will associate to each word, a list tuple (document, tf, idf), with all the documents containing that word. We decided to store tf and idf separately instead of directly storing the tfidf since, when calculating the cosine similarity, we will need the idf for each word -> document association.

Before creating the index we are going to parse the text (intro + plot) of each document and store all of them in a json file for easy and fast access in the future operations.

In [1]:
def save_all_docs_text(tsv_filepath, texts_filepath):
    """
    Creates a json file which contains the association tsv_filename -> text (intro + plot).
    Thanks to this file every time that a text is needed it is already stored into this json file.
    :param tsv_filepath:
    :param texts_filepath:
    :return:
    """
    files = os.listdir(tsv_filepath)
    d = dict()

    for f in files:
        print(f)
        d[f] = get_text(tsv_filepath, f)

    with open(texts_filepath, 'w') as out:
        json.dump(d, out)

In [ ]:
index_utils.save_all_docs_text(tsv_filepath, texts_filepath)

To retrive the file created we will use the following function

In [ ]:
texts_filepath = data_path + r'\texts.json'
texts = get_texts_file(texts_filepath)

Here is the function to calculate tf, idf and to create the scored index:

In [4]:
def tf(word, document):
    """
    Computes the term frequency (tf) of word into document.
    :param word: string
    :param document: string
    :return: tf
    """
    occurrences = document.count(word)
    return occurrences / len(document.split())

In [5]:
def idf(N, df):
    """
    Computes the inverse document frequency of a word.
    :param N: total number of documents
    :param df: number of documents which contain the word in question
    :return: idf
    """
    return math.log10(N / (df + 1))

In [ ]:
def create_scored_index(index, N, texts, partial_indexes, i):
    """
    Iterates over the index (portion of the whole inverted_index and, for each word of the index,
    computes the tfidf of each document associated to the word.
    :param index: inverted index in json format
    :param N: total number of documents
    :param texts: dict with associations doc_name -> text (intro + plot)
    :param partial_indexes: list of partial indexes (used for the multithread operations)
    :param i: index of the actual partial index
    :return: None
    """
    scored_index = {}
    count = 1
    for word, documents in index.items():
        print(count)
        scored_index[word] = []
        df = len(documents)
        for doc_name in documents:
            text = texts[doc_name]
            _tf = index_utils.tf(word, text)
            _idf = index_utils.idf(N, df)
            scored_index[word].append((doc_name, _tf, _idf))
        count += 1

    partial_indexes[i] = scored_index

We can see that the functions stores the scored index just created as element of a list. This is beacause, as for the tsv file creation, this process gets handled in multithread. This allows the index creation process to be much faster. Here is how:

In [2]:
def multithread_create_scored_index(_index, scored_index_filepath, N, texts, n_threads):
    """
    This functions operates in multithread. It subdivides the items of the initial inverted index
    into chucks depending on the number of threads to use and let each thread call the function
    create_scored_index over each chunk.
    :param _index:
    :param scored_index_filepath:
    :param N:
    :param texts:
    :param n_threads:
    :return:
    """
    # calculate the number of chunks and subdivide the initial index
    l_chunks = len(_index) // n_threads
    items = list(_index.items())
    chunks = [dict(items[i:i + l_chunks]) for i in range(0, len(_index), l_chunks)]

    if len(chunks) > n_threads:
        chunks[-2].update(chunks[-1])
        del chunks[-1]

    threads = []
    partial_indexes = [{} for i in range(n_threads)]  # each partial scored_index will be saved in this list

    start_time = time.time()
    for i in range(n_threads):
        # create the threads with the target function create_scored_index and the parameters.
        threads.append(Thread(name='Thread-' + str(i),
                              target=index.create_scored_index,
                              args=[chunks[i], N, texts, partial_indexes, i]))

    # start each thread
    for t in threads:
        t.start()

    # wait for each thread to finish its execution
    for t in threads:
        t.join()

    # join all the partial scored indexes into a unique index
    scored_index = dict()
    for d in partial_indexes:
        scored_index.update(d)

    # remove eventual duplicated elements
    for word, lst in scored_index.items():
        lst = list(set(lst))
        scored_index[word] = lst

    with open(scored_index_filepath, 'w') as out:
        json.dump(scored_index, out)

    minutes = (time.time() - start_time) / 60
    print("---------------- SCORED INDEX created in %s minutes ------------------" % minutes)

Running the next section the scored inverted index gets created.

In [ ]:
scored_index_filepath = data_path + r'\scored_index.json'
N = len(os.listdir(movies_path)) # number of total documents
_index = index_utils.get_index(index_filepath)
texts = get_texts_file(texts_filepath)
multithread_create_scored_index(_index, scored_index_filepath, N, texts, n_threads=6)

#### 2.2.2) Execute the query

Now that we have the scored index we can define the functions for the execution of the query.  
In this section, Cosine Similarity is used to see how much the query is equal to the document found.

To calculate the Cosine Similarity between each document and the query we need to be able to compute the **dot product** between 2 list of floats and the **norm** of a list of floats.

In [6]:
def dot(a1, a2):
    """
    Given 2 lists of integers a1 and a2, performs the dot product between the 2.
    :param a1:
    :param a2:
    :return: dot product
    """
    s = 0
    if len(a1) != len(a2):
        raise Exception('Arrays have not the same length!')
    for i in range(len(a1)):
        s += a1[i] * a2[i]
    return s

In [7]:
def norm(a):
    """
    Given the list of integers a, performs the norm of the list.
    :param a:
    :return: norm of a
    """
    s = 0
    for num in a:
        s += (float(num) ** 2)
    return np.sqrt(s)

Now we are able to compute the cosine similarity between the query and each documents containing all the words in it.

In [8]:
def cosine_similarity(query, words_dict):
    """
    Given a query and a dict words_dict, which associates to each word a list of (doc, tf, idf) for that word,
    returns a dict doc_cosine which associates to each documents the cosine similarity to the query.
    :param query: string
    :param words_dict: dict
    :return: doc_cosine dict
    """
    words = query.split()

    # for each word in the query, calculate its term frequency and put the association in a dict.
    # then for each document containing the word, get the tf and idf and put it in new lists. In the end
    # associate the lists to the doc in new dicts doc_to_word.
    word_score = {}
    doc_to_word_tf = {}
    doc_to_word_idf = {}
    for word in words:
        word_score[word] = index_utils.tf(word, query)
        for (doc, _tf, _idf) in words_dict[word]:
            l = doc_to_word_tf.get(doc, [])
            l.append(_tf*_idf)
            doc_to_word_tf[doc] = l
            l1 = doc_to_word_idf.get(doc, [])
            l1.append(_idf)
            doc_to_word_idf[doc] = l1

    # for each association (doc, tfidf_vec) in the doc_to_word dict, compute the cosine similarity
    # between the doc and the query, given the vector containing the tf's of each word in the query.
    doc_cosine = {}
    query_tf_vec = list(word_score.values())
    for (doc, tfidf_vec) in doc_to_word_tf.items():
        query_tfidf_vec = [query_tf_vec[i]*doc_to_word_idf[doc][i] for i in range(len(query_tf_vec))]
        doc_cosine[doc] = dot(tfidf_vec, query_tfidf_vec) / (norm(query_tfidf_vec) * norm(tfidf_vec))

    return doc_cosine

Now let's define some functions the we will need for the execution of the query search.

In [9]:
def remove_elements(lst, k):
    """
    Given in input the list lst, removes the elements that are repeated less than k times.
    :param lst:
    :param k:
    :return:
    """
    counted = Counter(x[0] for x in lst)

    temp_lst = []
    for el in counted:
        if counted[el] < k:
            temp_lst.append(el)

    res_lst = []
    for (el, tf, idf) in lst:
        if el not in temp_lst and el not in res_lst:
            res_lst.append(el)

    return res_lst

In [10]:
def get_top_items(results, k):
    """
    Takes in input a dictionary of results (index: list of info) and an integer k > 0.
    Orders the items of the dict using a heap structure and returns a list with the first k movies.
    :param results: dict
    :param k: int
    :return: dict
    """
    from heapq import _heapify_max

    # put every items in a tuple (similarity, index) and append to a new list
    lst = []
    for (index, info) in results.items():
        lst.append((info[-1], index))

    # order the list using an heap data structure and take the first k items
    ordered_list = []
    for i in range(k):
        try:
            _heapify_max(lst)
            ordered_list.append(lst[0])
            lst.pop(0)
        except: # if there are less than k items, just keep the ones found
            break

    # put the items in a new list and return it
    best_items = []
    for sim, index in ordered_list:
        best_items.append(results[index])

    return best_items

And here is the function for the query execution

In [11]:
def execute_query_cosine(index, query, index_to_link_path, tsv_path, k):
    """
    This function takes in input a scored_index and a query and prints the 5 movies
    with most similarity to the query.
    :param index:
    :param query:
    :return: dataframe with results
    """
    # Creating a set of movies which contains all the words form the query
    movies = []
    words_dict = dict()  # associates to each word the list of (doc, score)
    words = query.split()
    n_words = len(words)
    for word in words:
        movies.extend(index.get(word, []))
        words_dict[word] = index.get(word, [])

    # keep only the movies that contain all the words from the query
    relevant_movies = utils.remove_elements(movies, n_words)
    for (word, lst) in words_dict.items():
        temp = []
        for (doc, tf, idf) in lst:
            if doc in relevant_movies:
                temp.append((doc, tf, idf))
        words_dict[word] = temp

    # calculate the cosine similarity between each movie and the query
    # and store it in a dict
    cosine_similarities = similarities_utils.cosine_similarity(query, words_dict)

    with open(index_to_link_path, 'r') as f:
        index_to_link = json.load(f)

    # create the final dict containing title, intro, url and similarity for each movie
    results = dict()
    for movie in relevant_movies:
        filepath = tsv_path + '/' + str(movie)
        tsv = pd.read_csv(filepath, delimiter='\t')
        # with open(filepath, 'r') as f:
        #     tsv = csv.DictReader(f, dialect='excel-tab')
        title = tsv.loc[0, 'title']
        intro = tsv.loc[0, 'intro']
        index = re.findall(r'\d+', movie)[0]
        url = index_to_link[str(index)]
        similarity = round(cosine_similarities[movie], 2)
        results[index] = [title, intro, url, similarity]

    # get the first k items with most similarity to the query
    first_items = utils.get_top_items(results, k)

    # convert the dict to a pandas dataframe
    table = pd.DataFrame(first_items, columns=['Title', 'Intro', 'Wikipedia Url', 'Similarity'])
    return table

Run the next section to input and execute a query

In [ ]:
query = utils.clean_up(input("Please, write a query: "))
k = -1
while k < 0:
    k = int(input("Choose the maximum number of results to show: "))
index = index_utils.get_index(scored_index_filepath)
print("Cosine Similarity Results")
table = execute_query_cosine(index, query, index_to_link_path, tsv_filepath, k)
print(table.to_string(index=False))

## 3. Define a new score!

### How to sort them?

Given the relevants movies, to define a new score we work in terms of sets. First of all create a dataframe containing all the variables you need (title, intro, plot, language, producer, etc..). 

Then, given a query, see if in each variable (for each film) there is the required word in this way:
\begin{equation*}
\frac {A \cap B}{A \cup B}
\end{equation*}
Where A is a set composed of the words of the query, while B is the set of words in each cell of the dataframe.

This is called Jaccard index and is between 0 and 1.

Now, for each cell of the dataframe there is a Jaccard index  which gives us a 'frequency' of the single words of the query for each cell of the dataframe. Then do a sum of each row (that contains all the jaccard indeces for a film) to see a new index between 0 and the numbers of variables. If this new score is highest then the film is the most relevant.

The last step is to normalize the array containing the new score between 0 and 1. In this way the closest film to 1 will be the most similar to the query among the relevant films, the one closest to 0 will be the least similar among the films relevant (does not therefore mean that it is not relevant).

This algorithm works because if in the query there is a word of the title, automatically that film will be among the first on the list because the Jaccard index will be high.
Similarly, if there is a date it will be particularly important because the 'date' cell contains few words.
So if it finds a query word in cells that contain many words it tends to take it very little into consideration, as opposed to cells with few words that (rightly) have more importance for a search.

#### Algorithm

Create a dataframe with only the movies that contain all the words from the query

In [13]:
def get_dataframe(relevant_movies, tsv_filepath, index_to_link_filepath):
    cols = ['title', 'intro', 'plot', 'director', 'producer', 'writer', 'starring', 'music', 'release date', 'runtime',
            'country', 'language', 'budget', 'url']
    final_df = pd.DataFrame(columns=cols)

    with open(index_to_link_filepath, 'r') as f:
        index_to_link = json.load(f)

    for movie in relevant_movies:
        temp_df = pd.read_csv(tsv_filepath + '/' + movie, delimiter='\t')
        index = re.findall(r'\d+', movie)[0]
        url = index_to_link[str(index)]
        temp_df['url'] = [url]
        final_df = pd.concat([final_df, temp_df], sort=True)

    return final_df.reset_index(drop=True)

Calculate an array containing the sum of the Jaccard indeces for each relevant film.

In [14]:
# Jaccard similarity give us an index between 0 and 1
def jaccard_similarity(_query, _text):
    _query = set(_query.split())
    _text = set(_text.split())
    return len(_query.intersection(_text)) / len(_query.union(_text))

Then normalize it between 0 and 1.

In [15]:
def normalize(values, actual_bounds, desired_bounds):
    return [desired_bounds[0] + (x - actual_bounds[0]) * (desired_bounds[1] - desired_bounds[0]) / (actual_bounds[1] - actual_bounds[0]) for x in values]

Here is defined the function to execute the query:

In [16]:
def execute_query_jaccard(index, query, index_to_link_path, tsv_filepath, k):
    # Creating a set of movies which contains all the words form the query
    movies = []
    words_dict = dict()  # associates to each word the list of (doc, score)
    words = query.split()
    n_words = len(words)
    for word in words:
        movies.extend(index.get(word, []))
        words_dict[word] = index.get(word, [])

    # keep only the movies that contain all the words from the query
    relevant_movies = utils.remove_elements(movies, n_words)

    dataframe = similarities_utils.get_dataframe(relevant_movies, tsv_filepath, index_to_link_path)

    jac_score_final = []
    # Jaccard index for each position
    cols = dataframe.columns
    for index, row in dataframe.iterrows():
        jac_score = []
        for col in cols:
            text = str(row[col])
            jac_score.append(similarities_utils.jaccard_similarity(query, utils.clean_up(text)))
        jac_score_final.append(sum(jac_score))

    if len(jac_score_final) > 1:
        _jac = similarities_utils.normalize(jac_score_final,
                                            (min(jac_score_final), max(jac_score_final)),
                                            (0, 1))
    else:
        _jac = jac_score_final

    dataframe['similarity'] = list(map(lambda x: round(x, 2), _jac))

    results = dict()
    for index, row in dataframe.iterrows():
        title = row['title']
        intro = row['intro']
        url = row['url']
        sim = row['similarity']
        results[index] = [title, intro, url, sim]

    # get the first k items with most similarity to the query
    first_items = utils.get_top_items(results, k)

    # convert the dict to a pandas dataframe
    table = pd.DataFrame(first_items, columns=['Title', 'Intro', 'Wikipedia Url', 'Similarity'])
    return table

Run the next section to input and execute a query

In [ ]:
query = utils.clean_up(input("Please, write a query: "))
k = -1
while k < 0:
    k = int(input("Choose the maximum number of results to show: "))
index = index_utils.get_index(scored_index_filepath)
print("Jaccard Similarity Results")
table = execute_query_jaccard(index, query, index_to_link_path, tsv_filepath, k)
print(table.to_string(index=False))

### Professionality is the key!

To be a little bit more professional, here is a section of code that permits the user to choose the desidered type of search engine.

In [ ]:
    print("\nChoose between the following search engines:")
    print("1 - Base Search Engine")
    print("2 - Search Engine based on Cosine Similarity")
    print("3 - Search Engine based on Jaccard Similarity")
    choice = int(input("Enter your choice: "))
    while choice not in [1, 2, 3]:
        print("Please, insert a valid choice!")
        choice = int(input("Enter your choice: "))

    query = utils.clean_up(input("Please, write a query: "))

    if choice == 1:
        index = index_utils.get_index(index_filepath)
        table = execute_query_base(index, query, index_to_link_path, tsv_filepath)
    else:
        k = -1
        while k < 0:
            k = int(input("Choose the maximum number of results to show: "))
        index = index_utils.get_index(scored_index_filepath)
        if choice == 2:
            print("Cosine Similarity Results")
            table = execute_query_cosine(index, query, index_to_link_path, tsv_filepath, k)
        elif choice == 3:
            print("Jaccard Similarity Results")
            table = execute_query_jaccard(index, query, index_to_link_path, tsv_filepath, k)
    print(table.to_string(index=False))

## 4. Algorithmic question

The function take in input three values: string, first value and last value. If the string is composed just by one value then the longest substring palindrome will be 1. If the string is composed by two values and these values are equal, then the longest substring palindrome will be 2.

For strings with a length bigger than 2 is used a recursive function. If first and last value are equal then recall the function and move the first value to the second and the last value to the previous one. Also add 2 to the final result as the values are the same

If those values aren't equal, the same function is recalled, first moving the last value from the last current to the previous one, then trying moving the first value from the first current to the next. So take the biggest result from those recursive function.

In [17]:
def max_palindrome(word, i, k):  # i=word[0]  k=word[-1]

    if (i == k):  # len(word)==1
        return 1

    if (word[i] == word[k] and i + 1 == k):  # len(word)==2 and word[0]==word[1]
        return 2

    if (word[i] == word[k]):                        # word[0]==word[-1]
        return max_palindrome(word, i + 1, k - 1) + 2  # recall the function and
                                                       # add 2 for count the previous letters that are equal

    res1 = max_palindrome(word, i, k - 1)
    res2 = max_palindrome(word, i + 1, k)

    if res1 > res2:
        return res1
    else:
        return res2

In [22]:
print('Insert a sequence of characters: ')
word = input().strip()
n = len(word)
print("Max palindrome subsequence has length: ",
      max_palindrome(word, 0, n - 1))

Insert a sequence of characters: 
itopinonavevanonipoti
Max palindrome subsequence has length:  21
